# Working with MANTO data

One of the goals of DICES is to be interoperable with other Digital Classics projects. In this notebook, we connect to [MANTO](https://www.manto-myth.org/manto), a database of people in Greek and Roman mythology. Many of our DICES characters have records in MANTO, and MANTO provides a lot of potentially useful information about them that's beyond the scope of DICES.

For example, MANTO gives information about relationships between characters, connections to places in the Greco-Roman world, and primary sources that attest these attributes.

Example:
- the MANTO record for Aphrodite: https://resource.manto.unh.edu/8182231

Note that not all of our characters have MANTO records: some of our characters aren't mythological, and in some cases we haven't finished updating our character identifiers to sync with MANTO.

### Preliminaries

In [ ]:
# Google Colab only:
#   run the line below to install the DICES client

!pip install --quiet git+https://github.com/cwf2/dices-client.git

#### Import statements 

Here we load DICES code: the MANTO tools are in the `manto` module. We're also using Pandas for tabular data.

In [1]:
from dicesapi import DicesAPI, SpeechGroup
from dicesapi import manto
import pandas as pd

Create a connection to DICES

In [2]:
api=DicesAPI(logfile='dices.log', logdetail=0)

Instantiate a DICES character to get started

In [3]:
char = api.getCharacters(name='Telemachus')[0]
print(char)

<Character 938: Telemachus>


### Retrieving MANTO entities

Let’s create a new MANTO entity by retrieving Telemachus’ ID from MANTO.

In [4]:
manto_ent = manto.getMantoChar(char)
print(manto_ent)

<MANTO Entity 8190074: 👤 Telemachos>


### Using MANTO ties to find relatives

This code is still in development and subject to change. But for now, this is how we search MANTO for Telemachus' parents. Under the hood, we're checking two different MANTO ties, `31764`, "son of", and `31765`, "daughter of."

This function currently returns MANTO Entity objects.

In [5]:
parents = manto_ent.getParents()
print(parents)

[<MANTO Entity 8189173: 👤 Penelope>, <MANTO Entity 8189902: 👤 Odysseus>]


Let's look for grandparents. Note that when sources disagree about parentage, MANTO returns an inclusive list.

In [6]:
for p in parents:
    print(f'Grandparents on {p.name}’s side:')
    for gp in p.getParents():
        print(' - ' + gp.name)

Grandparents on 👤 Penelope’s side:
 - 👤 Icarios (Spartan)
 - 👤 Periboia (Spartan)
 - 👤 Polycaste (Acarnanian)
Grandparents on 👤 Odysseus’s side:
 - 👤 Anticleia (Ithacan)
 - 👤 Laertes
 - 👤 Sisyphos


### Boolean relationship checker

#### For MANTO Entities

Sometimes we just want a yes/no answer as to parentage. For MANTO Entities, methods like `isChildOf()` will take a second Manto entity as argument and return `True` or `False`. This can be useful, for example, for filtering a list.

In [7]:
# a list of character pairs, by MANTO ID
pairs = [
    ('8182179', '8182084'),  # Anchises and Aeneas
    ('8188282', '8190182'),  # Eteocles and Oedipus
    ('8189802', '8182035'),  # Menelaus and Agamemnon
]

for a, b in pairs:
    ent_a = manto.getMantoID(a)
    ent_b = manto.getMantoID(b)
    print(f'Is {ent_a.name} the child of {ent_b.name}? ', ent_a.isChildOf(ent_b))

Is 👤 Anchises (Trojan) the child of 👤 Aineias?  False
Is 👤 Eteocles (Theban) the child of 👤 Oedipus?  True
Is 👤 Menelaos the child of 👤 Agamemnon?  False


#### For DICES Characters

For convenience, there are also wrapper functions that perform the same checks on DICES Characters by resolving the characters to MANTO entities first, then calling the boolean methods above. If you pass a CharacterInstance instead of a Character, the function will try to determine the MANTO ID of the underlying Character.

If passed a Character with no MANTO ID, or an anonymous CharacterInstance with no Character, these functions return `None`. You can use the optional `err_val` parameter to set a different fallback value; for example, if you're using the function in a filter and want to ensure it always returns a Boolean, set this to `False`.

For these next examples, let's start with a list of Telemachus' interlocutors.

In [8]:
# download all speeches by Telemachus
speeches = api.getSpeeches(spkr_id=char.id, progress=True)
print(f'{len(speeches)} speeches')

# extract the list of addressees
addressees = speeches.getAddrs(flatten=True)
print(f'{len(addressees)} unique addressees.')

84 speeches
22 unique addressees.


In this block, we use call `charIsChild()` on the speaker and his addressees to test parentage using MANTO's data. Where the function returns `None`, we know data was missing.

In [9]:
for addr in addressees:
    check = manto.charIsChild(char, addr)
    if check is not None:
        print(f'Is {char.name} the child of {addr.name}?', check)

Is Telemachus the child of Menelaus? False
Is Telemachus the child of Nestor? False
Is Telemachus the child of Penelope? True
Is Telemachus the child of Ctesippus? False
Is Telemachus the child of Athena-Mentes? False
Is Telemachus the child of Euryclea? False
Can't get MANTO data: <CharacterInstance 153: all present> is an anonymous CharacterInstance
Is Telemachus the child of Eumaeus? False
Is Telemachus the child of Odysseus? True
Is Telemachus the child of Aegyptius? False
Can't get MANTO data: <CharacterInstance 142: suitors of Penelope> is an anonymous CharacterInstance
Can't get MANTO data: <CharacterInstance 167: women> is an anonymous CharacterInstance
Is Telemachus the child of Peisistratus? False
Is Telemachus the child of Agelaus? False
Is Telemachus the child of Antinous? False
Is Telemachus the child of Athena? False
Is Telemachus the child of Theoclymenus? False
Is Telemachus the child of Athena-Mentor? False
Is Telemachus the child of Helena? False
Is Telemachus the chi

Or we can use MANTO ties in a custom filter. In this case I'll silence the debugging messages and force `charIsChild()` to return `False` if no MANTO ID is found.

In [10]:
# filter a CharacterInstanceGroup based on MANTO ties:
parent_addrs = addressees.advancedFilter(lambda addr: manto.charIsChild(char, addr, debug=False, err_val=False))

print(f'Parent addressees of {char.name}:')
for p in parent_addrs:
    print(p.name)
print()
    
# now use the filtered list of CharacterInstances to filter the original SpeechGroup
parent_speeches = speeches.filterAddrInstances(parent_addrs)

print(f'Speeches in which {char.name} addresses a parent:\n')
for s in parent_speeches:
    print(f'{s.author.name} {s.work.title} {s.l_range}\t{s.getAddrString()}')

Parent addressees of Telemachus:
Penelope
Odysseus
Odysseus-beggar

Speeches in which Telemachus addresses a parent:

Homer Odyssey 1.346-1.359	Penelope
Homer Odyssey 16.44-16.45	Odysseus-beggar
Homer Odyssey 16.113-16.134	Odysseus-beggar
Homer Odyssey 16.181-16.185	Odysseus
Homer Odyssey 16.194-16.200	Odysseus
Homer Odyssey 16.222-16.224	Odysseus
Homer Odyssey 16.241-16.257	Odysseus
Homer Odyssey 16.263-16.265	Odysseus
Homer Odyssey 16.309-16.320	Odysseus-beggar
Homer Odyssey 17.46-17.56	Penelope
Homer Odyssey 17.108-17.149	Penelope
Homer Odyssey 18.61-18.65	Odysseus-beggar
Homer Odyssey 18.227-18.242	Penelope
Homer Odyssey 19.36-19.40	Odysseus-beggar
Homer Odyssey 20.262-20.267	Odysseus, suitors of Penelope
Homer Odyssey 21.344-21.353	Penelope
Homer Odyssey 22.101-22.104	Odysseus
Homer Odyssey 22.154-22.159	Odysseus
Homer Odyssey 22.356-22.360	Odysseus
Homer Odyssey 23.97-23.103	Penelope
Homer Odyssey 23.124-23.128	Odysseus
Homer Odyssey 24.511-24.512	Odysseus
